Crear imagenes binarias con las lesiones para cada imagen del ddsm. 
Ahora si hay avias lesiones tenemos una mascara por lesion

In [1]:
import pathlib
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ddsm_root = pathlib.Path('/home/alalbiol/Data/mamo/DDSM_png_16bit_1152x896')
png_images = [f for f in ddsm_root.glob('**/*.png') if "_mask_" not in f.name]
print("Number of images:", len(png_images))


def get_mask_for_image(image_path):
    mask_paths = [f for f in ddsm_root.glob('**/*.png') if "_mask_" in f.name and image_path.stem in f.name]
    
    masks = []
    if not mask_paths:
        return None
    
    for mask_path in mask_paths:
        #print(mask_path)
        mask = np.array(Image.open(mask_path)).astype(np.uint8) / 255
        if "MASS_BENIGN" in mask_path.name:
            mask = mask * 1
        elif "CALCIFICATION_BENIGN" in mask_path.name:
            mask = mask * 2
        elif "MASS_MALIGNANT" in mask_path.name:
            mask = mask * 3
        elif "CALCIFICATION_MALIGNANT" in mask_path.name:
            mask = mask * 4
        else:
            mask = mask * 5
        
         
        masks.append(mask)
    mask = np.array(masks).sum(axis=0).astype(np.uint8)
    return mask


                         
                         

Number of images: 11023


In [27]:
from tqdm import tqdm
for png_image in tqdm(png_images):
    mask = get_mask_for_image(png_image)
    if mask is None:
        #print(f"No mask found for {png_image}")
        continue
    
    #print(mask.dtype, mask.shape, mask.min(), mask.max())
    mask_filename = png_image.stem + "_totalmask.png"
    mask_path = png_image.parent / mask_filename
    mask = Image.fromarray(mask)
    mask.save(mask_path)


100%|██████████| 11023/11023 [21:08<00:00,  8.69it/s]
